In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [17]:
!pip install wxee

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.1 MB/s eta 0:00:00


In [2]:
# !earthengine authenticate
import ee
import os

# Set the path to the service account key file
service_account = 'editor@ee-manmeet20singh15-wbis.iam.gserviceaccount.com'
key_file = 'drive/MyDrive/earth_engine/ee-manmeet20singh15-wbis-fab7f1ca35e0.json'

# Use the service account for authentication
credentials = ee.ServiceAccountCredentials(service_account, key_file)
ee.Initialize(credentials)

In [16]:
# Load the GFS dataset
data_collection = ee.ImageCollection("NOAA/GFS0P25");

# Define the initialization date
forecast_date = ee.Date('2019-03-04') # Initialization time

# Define lead day (e.g., Lead Day 2)
lead_days = 2;

# Get forecast data for Lead Day 2
forecast_lead_day = data_collection.filterDate(forecast_date, forecast_date.advance(6, 'hour')) \
                    .filter(ee.Filter.gte('forecast_time', forecast_date.advance(lead_days, 'day').millis())) \
                    .filter(ee.Filter.lt('forecast_time', forecast_date.advance(lead_days + 1, 'day').millis()))

#print('Lead Day 2 Forecast:', forecast_lead_day2)

In [18]:
# Define the center coordinates of Austin, Texas
centerLat = 30.2672 # Latitude of Austin
centerLon = -97.7431 # Longitude of Austin

# Define the bounding box size (adjust as needed)
latDiff = 0.2 # Latitude range (degree buffer)
lonDiff = 0.2 # Longitude range (degree buffer)

# Create an AOI (Bounding Box) around Austin
aoi = ee.Geometry.Polygon([
  [
    [centerLon - lonDiff, centerLat - latDiff], #// Southwest corner
    [centerLon + lonDiff, centerLat - latDiff], #// Southeast corner
    [centerLon + lonDiff, centerLat + latDiff], #// Northeast corner
    [centerLon - lonDiff, centerLat + latDiff]  #// Northwest corner
  ]
])

In [19]:
import wxee
ds_gfs = forecast_lead_day.wx.to_xarray(region=aoi.bounds(), scale=27830)

Requesting data:   0%|          | 0/24 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/wxee/utils.py:149: UserWarning: Different non-null values were encountered for the same variable at the same time coordinate. The first value was taken.
  warnings.warn(


In [20]:
ds_gfs

<xarray.Dataset> Size: 336B
Dimensions:                               (time: 1, x: 2, y: 2)
Coordinates:
  * time                                  (time) datetime64[ns] 8B 2019-03-04
  * x                                     (x) float64 16B -97.88 -97.63
  * y                                     (y) float64 16B 30.38 30.13
    spatial_ref                           int64 8B 0
Data variables:
    temperature_2m_above_ground           (time, y, x) float64 32B 7.535 ... ...
    specific_humidity_2m_above_ground     (time, y, x) float64 32B 0.002476 ....
    relative_humidity_2m_above_ground     (time, y, x) float64 32B 38.02 ... ...
    u_component_of_wind_10m_above_ground  (time, y, x) float64 32B -2.913 ......
    v_component_of_wind_10m_above_ground  (time, y, x) float64 32B -2.086 ......
    total_precipitation_surface           (time, y, x) float64 32B 0.0 ... 0.0
    precipitable_water_entire_atmosphere  (time, y, x) float64 32B 7.9 ... 8.1
    total_cloud_cover_entire_atmosphere   (time, y, x) float64 32B 79.0 ... 51.0
    downward_shortwave_radiation_flux     (time, y, x) float64 32B 608.0 ... ...
Attributes:
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    AREA_OR_POINT:           Area
    _FillValue:              -32768.0
    scale_factor:            1.0
    add_offset:              0.0